In [7]:
import numpy as np
from src.PLS.baseSIR import model_sir
import pickle
from src.PLS.metaSIR import basic_square_map
from src.PLS.metaSIR import straight_line_distances
from src.PLS.metaSIR import meta_model_sir
from src.PLS.metaSIR import basic_kernel

In [8]:
Typ = ["simple","simple_I0","meta"][2]
siz = ["10","100","1000"][0]
seed =1912

In [9]:
#import distributions
distro = np.genfromtxt(f"../../Data/R0_3/Sims_SIR/Sampled_distros/Distro_{Typ}_{siz}.csv",delimiter=",")

In [10]:
#setup dictionary
output_dictionary = {}
output_dictionary["Final_size"] = []
output_dictionary["Extinction"] = []
output_dictionary["Peak"] = []
output_dictionary["Tmax"] = []
output_dictionary["Tpeak"] = []

In [11]:
#Scen Toggle
if Typ == "simple":
    beta_distro = distro
    n_sims = len(beta_distro)
    gamma = 1
    mu = 0
    tstep = 0.05
    tmax = 100
    rng = np.random.default_rng(seed)
    div=1
    if siz == "1000":
        X0 = [900,100,0]
    elif siz == "100":
        X0 = [90,10,0]
    elif siz == "10":
        X0 = [9,1,0]
    else:
        raise ValueError("Not possible! Choose from 1000,100,10")
    #Run Sims
    for i in range(n_sims):
        beta = beta_distro[i]/div
        rinf, ex,t,peak,peak_t  = model_sir(X0,mu,beta,gamma,tmax,tstep,rng)
        output_dictionary["Final_size"].append(rinf)
        output_dictionary["Extinction"].append(ex)
        output_dictionary["Peak"].append(peak)
        output_dictionary["Tmax"].append(t)
        output_dictionary["Tpeak"].append(peak_t)
elif Typ == "simple_I0":
    beta_distro = distro
    n_sims = len(beta_distro)
    gamma = 1
    mu = 0
    tstep = 0.05
    tmax = 100
    rng = np.random.default_rng(seed)
    div=1
    if siz == "1000":
        X0 = [999,1,0]
    elif siz == "100":
        X0 = [99,1,0]
    elif siz == "10":
        X0 = [9,1,0]
    else:
        raise ValueError("Not possible! Choose from 1000,100,10")
    #Run Sims
    for i in range(n_sims):
        beta = beta_distro[i]/div
        rinf, ex,t,peak,peak_t  = model_sir(X0,mu,beta,gamma,tmax,tstep,rng)
        output_dictionary["Final_size"].append(rinf)
        output_dictionary["Extinction"].append(ex)
        output_dictionary["Peak"].append(peak)
        output_dictionary["Tmax"].append(t)
        output_dictionary["Tpeak"].append(peak_t)
elif Typ == "meta":
    beta_distro = distro
    n_sims = len(beta_distro)
    N = 4
    test_map = basic_square_map(N)
    test_distances = straight_line_distances(test_map,N,scaling=10)
    gamma = 1
    rng = np.random.default_rng(seed)
    tstep = 0.01
    tmax = 100
    if siz == "1000":
        X0 = [[999,1,0]]
        div = sum(X0[0])
        for i in range(N-1):
            X0.append([1000,0,0])
    elif siz == "100":
        X0 = [[99,1,0]]
        div = sum(X0[0])
        for i in range(N-1):
            X0.append([100,0,0])
    elif siz == "10":
        X0 = [[9,1,0]]
        div = sum(X0[0])
        for i in range(N-1):
            X0.append([10,0,0])
    #Run Sims
    for i in range(n_sims):
        beta = beta_distro[i]/div
        rinf, ex,t  = meta_model_sir(X0,beta,gamma,N,test_distances,basic_kernel,tmax,tstep,rng)
        output_dictionary["Final_size"].append(rinf)
        output_dictionary["Extinction"].append(ex)
        #output_dictionary["Peak"].append(peak)
        output_dictionary["Tmax"].append(t)
        #output_dictionary["Tpeak"].append(peak_t)

In [12]:
#Pickle Sims
with open(f"../../Data/R0_3/Sims_SIR/Pickles/Sim_dict_{Typ}_{siz}.pickle", 'wb') as handle:
    pickle.dump(output_dictionary,handle, protocol=pickle.HIGHEST_PROTOCOL)